### MCOE Compilation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pudl
import pandas as pd
import sqlalchemy as sa
import sys
import logging
import pathlib

In [3]:
# this basic setup for logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]
pd.options.display.max_columns = None

In [4]:
# pudl_settings is a dictionary that includes the paths to several key pudl directories
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])

In [5]:
# the creation of the pudl_out object
# this will compile output tables like mcoe - if you want to restart/wipe the saved elements rerun this cell
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine, freq='AS', rolling=True)

In [6]:
# run mcoe method... the original function is in pudl.analysis.mcoe.mcoe()
mcoe = pudl_out.mcoe()

filling in fuel cost NaNs with rolling averages


In [7]:
# just an example...
mcoe[(mcoe['plant_id_eia'] == 6166)
     & (mcoe['report_date'].dt.year == 2016)
    ]

,report_date,plant_id_eia,plant_id_pudl,unit_id_pudl,generator_id,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,balancing_authority_code,balancing_authority_name,capacity_factor,capacity_mw,carbon_capture,cofire_fuels,current_planned_operating_date,deliver_power_transgrid,energy_source_code_1,energy_source_code_2,energy_source_code_3,energy_source_code_4,energy_source_code_5,energy_source_code_6,fuel_cost_per_mmbtu,fuel_cost_per_mwh,fuel_type_code_pudl,fuel_type_count,heat_rate_mmbtu_mwh,iso_rto_code,latitude,longitude,minimum_load_mw,multiple_fuels,nameplate_power_factor,net_generation_mwh,operational_status,operational_status_code,other_modifications_date,other_planned_modifications,ownership_code,planned_derate_date,planned_energy_source_code_1,planned_modifications,planned_net_summer_capacity_derate_mw,planned_net_summer_capacity_uprate_mw,planned_net_winter_capacity_derate_mw,planned_net_winter_capacity_uprate_mw,planned_new_capacity_mw,planned_new_prime_mover_code,planned_repower_date,planned_retirement_date,planned_uprate_date,retirement_date,startup_source_code_1,startup_source_code_2,startup_source_code_3,startup_source_code_4,state,summer_capacity_mw,summer_estimated_capability_mw,switch_oil_gas,syncronized_transmission_grid,technology_description,time_cold_shutdown_full_load_code,total_fuel_cost,total_mmbtu,turbines_inverters_hydrokinetics,turbines_num,uprate_derate_completed_date,uprate_derate_during_year,winter_capacity_mw,winter_estimated_capability_mw
170544,2016-01-01,6166,528,1.0,1,Rockport,9324,144,Indiana Michigan Power Co,PJM,"PJM Interconnection, LLC",0.460777,1300.0,False,False,None,None,SUB,BIT,None,None,None,None,2.654486,27.344364,coal,1,10.301190,PJM,37.9256,-87.0372,500.0,False,0.9,5261703.0,existing,OP,None,None,J,None,None,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,DFO,None,None,None,IN,1300.0,NaN,False,False,Conventional Steam Coal,OVER,1.438779e+08,5.420180e+07,None,NaN,None,False,1299.0,NaN
170545,2016-01-01,6166,528,2.0,2,Rockport,9324,144,Indiana Michigan Power Co,PJM,"PJM Interconnection, LLC",0.551367,1300.0,False,False,None,None,SUB,BIT,None,None,None,None,2.654486,26.965323,coal,1,10.158397,PJM,37.9256,-87.0372,500.0,False,0.9,6296167.0,existing,OP,None,None,J,None,None,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,DFO,None,None,None,IN,1300.0,NaN,False,False,Conventional Steam Coal,OVER,1.697782e+08,6.395896e+07,None,NaN,None,False,1299.0,NaN
